# Dask DataFrame

Материалы: 
* Макрушин С.В. Лекция 13: Dask DataFrame
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

In [1]:
import dask.dataframe as dd
import dask.bag as db
import pandas as pd
import datetime
import json
import dask
import re

In [2]:
path1 = "D:/FinUniver/Технологии обработки больших данных/Семинары/13_dask_dataframe/13_dask_dataframe_data/"
path2 = "D:/FinUniver/Технологии обработки больших данных/Семинары/12_dask_bag/12_dask_bag_data/"

## Задачи для совместного разбора

1. Считать данные из файлов в каталоге `accounts`. Содержат ли какие-либо из столбцов пропуски?

In [6]:
accounts = dd.read_csv(path1 + 'accounts.*.csv', dtype={'amount': 'float64'})
accounts

,id,names,amount
npartitions=3,,,
,int64,object,float64
,...,...,...
,...,...,...
,...,...,...


In [7]:
accounts.isna().sum().compute()

id        0
names     0
amount    3
dtype: int64

2. Подсчитать количество раз, которое то или иное имя встретилось в выборке. Вывести самое часто встречающееся имя.

In [8]:
accounts.groupby('names').size().compute()

names
Alice       185892
Bob         157065
Charlie     109236
Dan          73293
Edith        89991
Frank        99984
George      183249
Hannah      108632
Ingrid      132067
Jerry        96378
Kevin        84784
Laura        97216
Michael     148372
Norbert     188147
Oliver      121907
Patricia     62881
Quinn       118071
Ray         108610
Sarah       104781
Tim         162432
Ursula      108745
Victor      102656
Wendy       110657
Xavier       94445
Yvonne       61462
Zelda        89047
dtype: int64

In [13]:
accounts['names'].value_counts().compute()

Norbert     188147
Alice       185892
George      183249
Tim         162432
Bob         157065
Michael     148372
Ingrid      132067
Oliver      121907
Quinn       118071
Wendy       110657
Charlie     109236
Ursula      108745
Hannah      108632
Ray         108610
Sarah       104781
Victor      102656
Frank        99984
Laura        97216
Jerry        96378
Xavier       94445
Edith        89991
Zelda        89047
Kevin        84784
Dan          73293
Patricia     62881
Yvonne       61462
Name: names, dtype: int64

In [15]:
%%time
dask.compute(accounts.groupby('names')['amount'].sum(),
             accounts.groupby('names')['amount'].mean())

Wall time: 929 ms


(names
 Alice       121135049.0
 Bob         134707661.0
 Charlie     190678161.0
 Dan          67436523.0
 Edith        94317280.0
 Frank       132918185.0
 George      171385330.0
 Hannah      134076218.0
 Ingrid      162441094.0
 Jerry        49464228.0
 Kevin        92375351.0
 Laura       193055407.0
 Michael     392087039.0
 Norbert     142008672.0
 Oliver       48824444.0
 Patricia     58811563.0
 Quinn       145487072.0
 Ray          54871516.0
 Sarah        83719740.0
 Tim         165616261.0
 Ursula       36921307.0
 Victor       77636440.0
 Wendy        86795738.0
 Xavier      168807871.0
 Yvonne       24933113.0
 Zelda        62402294.0
 Name: amount, dtype: float64, names
 Alice        651.642077
 Bob          857.655499
 Charlie     1745.577526
 Dan          920.095002
 Edith       1048.074585
 Frank       1329.394553
 George       935.259292
 Hannah      1234.223967
 Ingrid      1229.990035
 Jerry        513.231526
 Kevin       1089.550393
 Laura       1985.839851
 Micha

3. Создать новую колонку, которая является результатом от деления значения `amount` нацело на 100, если `amount` > 100, и нулём в противном случае.

In [20]:
def f(x):
    if x > 100:
        return x // 100
    return 0

In [21]:
accounts['new_column'] = accounts['amount'].map(f)
accounts.head()

,id,names,amount,new_column
0,39,Victor,205.0,2.0
1,289,Hannah,87.0,0.0
2,234,Victor,1820.0,18.0
3,155,Ray,-600.0,0.0
4,266,Ray,572.0,5.0


## Лабораторная работа 13

__При решении данных задач не подразумевается использования других коллекций, кроме `dask.DataFrame`, если в задании явно не указано обратное.__

1. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты.

In [3]:
recipes = dd.read_csv(path1 + 'recipes_full_*.csv', 
                      parse_dates = ["submitted"],
                      dtype={'minutes': 'float64', 'n_steps': 'float64'})

2. Выведите метаинформацию о таблице: `npartitions` и типы столбцов.

In [4]:
recipes.npartitions

16

In [4]:
recipes

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
npartitions=16,,,,,,,,
,int64,object,float64,int64,datetime64[ns],float64,object,int64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


3. Выведите на экран 5 первых строк таблицы. Выведите на экран 5 последних строк таблицы. В случае сообщения об ошибки объясните причину и исправьте ошибку.

In [5]:
recipes.head(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10


In [50]:
recipes.tail(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
44011,1029131,tuti waffle snackies steakhouse,19.0,171345,1973-10-18,4.0,"according to a providence journal article, ama...",4
44012,1700703,noelias cheats crocante fleisch zitumbuwa,1.0,30228,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1
44013,1910650,rubbed restuffed pelmeni bedouin flavourful,60.0,591905,2009-09-26,3.0,another great recipe to add to the growing swe...,2
44014,713836,stems polpettine peezi,NaN,357389,2003-09-30,4.0,adapted from top secret recipes. love this!,9
44015,660699,clementines,64.0,29196,1973-06-03,6.0,this would make a great start to your holiday ...,8


4. Посчитайте, сколько строк содержит каждый из блоков (partitions).

In [49]:
recipes.map_partitions(len).compute()

0     237505
1      41450
2     235222
3      43733
4     238139
5      40816
6     234750
7      44205
8     237887
9      41068
10    234486
11     44468
12    238298
13     40656
14    234938
15     44016
dtype: int64

5. Найдите максимум в столбце `n_steps`. Визуализируйте граф вычислений для этой задачи. Прокомментируйте логику работы `dask` при решении данной задачи.

In [51]:
recipes['n_steps'].max().compute()

145.0

6. Посчитайте количество рецептов с группировкой по месяцам добавления отзыва в базу.

In [5]:
recipes_count = recipes.groupby(recipes['submitted'].dt.month).size().compute()
recipes_count

submitted
1     193363
2     173834
3     192389
4     186049
5     192487
6     184205
7     189337
8     187276
9     181081
10    187018
11    180974
12    183624
dtype: int64

7. Считайте файлы из архива `reviews_full.zip` (__ЛР12__) в виде `dask.bag`. Пользуясь результатом лабораторной работы 12, рассчитайте среднее значение оценок отзывов с группировкой по месяцам. После завершения всех вычислений преобразуйте результат к `pd.Series`.

In [6]:
def add_rating(record):
    pattern = re.compile(r'(?<=reviews_)[0-9]{1,}(?=\.json)')
    
    new_record = json.loads(record[0])
    new_record["rating"] = int(pattern.findall(record[1])[0])
    
    return new_record

In [7]:
reviews = db.read_text(path2 + 'reviews_*.json', include_path=True).map(add_rating).to_dataframe()
reviews['date'] = dd.to_datetime(reviews['date'])
reviews

,date,rating,recipe_id,review,user_id
npartitions=6,,,,,
,datetime64[ns],int64,int64,object,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [11]:
reviews

,date,rating,recipe_id,review,user_id
npartitions=6,,,,,
,datetime64[ns],int64,int64,object,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [12]:
reviews.head()

,date,rating,recipe_id,review,user_id
0,2016-05-08,0,292657,WOW!!! This is the best. I have never been abl...,452355
1,2006-06-14,0,433404,This was good but the dressing needed somethin...,329304
2,1985-11-19,0,2008187,"Very good,it was a hit for my family. I used 6...",227932
3,2019-05-21,0,270716,Made for ZWT-8 Family Picks after I saw these ...,171468
4,1972-09-18,0,1159916,Very nice slaw. I especially like that it does...,91392


In [13]:
mean_rating = reviews.groupby(reviews['date'].dt.month)['rating'].mean().compute()
mean_rating

date
1     4.408553
2     4.407135
3     4.406911
4     4.412259
5     4.415401
6     4.418419
7     4.411717
8     4.410822
9     4.410661
10    4.411452
11    4.408829
12    4.406908
Name: rating, dtype: float64

8. Пользуясь результатами решения задач 6 и 7, создайте `pd.DataFrame`, содержащий два столбца: `mean_rating`, `recipes_count`

In [14]:
result = pd.DataFrame({"recipes_count": recipes_count, "mean_rating": mean_rating})
result

,recipes_count,mean_rating
1,193363,4.408553
2,173834,4.407135
3,192389,4.406911
4,186049,4.412259
5,192487,4.415401
6,184205,4.418419
7,189337,4.411717
8,187276,4.410822
9,181081,4.410661
10,187018,4.411452
